# Text Generation using an LSTM in Keras
In this kernel you we will go over how to let a network create text in the style of sir arthur conan doyle. This kernel is heavily based on the [official keras text generation example](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py).  I also made [a video](https://youtu.be/QtQt1CUEE3w) on text generation using an LSTM network.
Content:
1. [Introduction](#1)
2. [Loading in data](#2)
3. [Preprocessing](#3)  
    3.1 [Map chars to integers](#3.1)  
    3.2 [Split up into subsequences](#3.2)
4. [Building model](#4)  
    4.1 [Helper Functions](#4.1)  
    4.2 [Defining callbacks and training the model](#4.2)
5. [Generate new text](#5)  
6. [Conclusion](#6)

<a id="1"></a> 
## 1. Introduction
Because the sequence in an text is important we will recurrent neural network which can remember its previous inputs.

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

<a id="2"></a> 
## 2. Loading in data

In [ ]:
text = open('../input/sherlock_homes.txt', 'r').read().lower()
print('text length', len(text))

In [ ]:
print(text[:300])

<a id="3"></a> 
## 3. Preprocessing

<a id="3.1"></a> 
### 3.1 Map chars to integers

Because we will be training on a character level we need to relate each unique character with a number.
We are going to create two dicts one from character to integer and one to transform back to character

In [ ]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

<a id="3.2"></a> 
### 3.2 Split up into subsequences
Creates an array of sentence data with the length maxlen as well as an array with the next character.

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

In [ ]:
print(sentences[:3])
print(next_chars[:3])

In [ ]:
# Print length
print(len(sentences))

We need to reshape our data in a format we can pass to the Keras LSTM The shape look like [samples, time steps, features]

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
print(x[:3])
print(y[:3])

<a id="4"></a> 
## 4. Building model
For this kernel I will use a really small LSTM network but if you want to get better results feel free to replace it with a bigger network.

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

<a id="4.1"></a> 
### 4.1 Helper Functions

I got this function from the lstm_text_generation example from keras. [https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)


Samples an index from a probability array with some temperature.

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Callback function to print predicted text generated by our LSTM. It prints generated text with 5 different temperatures [0.2, 0.5, 1.0, 1.2]. 0.2 will generate text with more ordinary word. 1.2 will generate wilder guesses.

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

<a id="4.2"></a> 
### 4.2 Defining callbacks and training the model

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [ ]:
callbacks = [print_callback, checkpoint, reduce_lr]

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

<a id="5"></a> 
## 5. Generate new text


We can generate text using the same approach as in the on_epoch_end helper function create by Keras.

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
print(generate_text(500, 0.2))

<a id="6"></a> 
## 6. Conclusion


After 5 epochs our LSTM performed a ok job and I'm more than satisfied with the result.

Here are a few things you can change to get better results

1. Add more LSTM Layers.
2. Use more LSTM Cells.
3. Train for more than 5 epochs. (25+)
4. Add dropout Layer.
5. Play around with the batch-size

